RNN(Recurrent Neural Network)
-------------------------

- 時系列を考慮した学習
- 代表的なものは自然言語処理がある
    - 他には過去の価格の推移から未来の価格の推移を予測するタスク、物体の動きを捉えた動画の情報からこれから先にどのように動くかを予測するタスクなど

### 自然言語
- 時系列の考慮が必要な学習として代表的なものに自然言語処理がある
- 自然言語では一般的に文章をそのまま扱うのではなく「形態素解析」を行い単語レベルで分割する
    - 時系列を考慮する処理においては、単語を前から後ろに順番に入力する

### LSTM(LongShortTermMemory)
- 勾配消失問題をCEC(ConstantErrorCarousel)とゲートという概念を導入することで解決したRNNの構造の一つ

### GRU(Gated Recurrent Unit)
- ゲートをアップデートゲート、リセットゲートに限定した構造

### TensorFlowでのRNN
- 1.tf.nn.rnn_cellパッケージのrnn_cell
- 2.tf.nnパッケージの○○_rnn

#### 構築手順
- 1.runn_cellを定義してラッピングなどをして各時間での挙動を定義
- 2.cellと入力データを入力して○○_rnnで順伝播
- 3.○○_rnnを加工して出力層を整形

Sequential MNIST
--------------------
- RNNを用いてMNISTの分類を行う

### サンプルソース
https://github.com/thinkitcojp/TensorFlowDL-samples/blob/master/s_mnist/mnist.py


In [3]:
# -*- coding:utf-8 -*-
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# mnistデータを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
# 入力データの整形
num_seq = 28
num_input = 28

# 実行時に画像データが渡される(1行1列の2階テンソル)
x = tf.placeholder(tf.float32, [None, 784])
# (バッチサイズ, 高さ, 幅)の3階テンソルに変換
input = tf.reshape(x, [-1, num_seq, num_input])
print(input)

Tensor("Reshape:0", shape=(?, 28, 28), dtype=float32)


In [5]:
# ユニット数128子のLSTMセル
# 3段に積む
stacked_cells = []
for i in range(3):
    stacked_cells.append(tf.nn.rnn_cell.LSTMCell(num_units=128))
    
# 配列にいれてラッパーを用いて多層にする
cell = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_cells)

# dynamic_rnnによる時間展開
# ※2度実行するとエラーになる
outputs, _ = tf.nn.dynamic_rnn(cell=cell, inputs=input, dtype=tf.float32)

In [6]:
# 最後の時間のTensorを取得
last_output = outputs[:, -1, :]

w = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
b = tf.Variable(tf.zeros([10]))

out = tf.nn.softmax(tf.matmul(last_output, w) + b)

In [7]:
#正解データの型を定義
y = tf.placeholder(tf.float32, [None, 10])
#誤差関数（クロスエントロピー）
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out), axis=[1]))

#訓練
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#評価
correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    #テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels

    for i in range(1000):
        step = i+1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images ,y:train_labels})

        #100ステップごとに精度を検証
        if step % 100 == 0:
            acc_val = sess.run( accuracy, feed_dict={x:test_images, y:test_labels})
            print('Step %d: accuracy = %.2f' % (step, acc_val))

Step 100: accuracy = 0.37
Step 200: accuracy = 0.51
Step 300: accuracy = 0.62
Step 400: accuracy = 0.72
Step 500: accuracy = 0.79
Step 600: accuracy = 0.83
Step 700: accuracy = 0.84
Step 800: accuracy = 0.85
Step 900: accuracy = 0.88
Step 1000: accuracy = 0.87


- 精度はそれほどよくない
- 精度の収束測度は普通のNNよりも遅い
    - 時系列情報分の層を積んでいるため